## Preparations

[Download and install SQLite](http://www.sqlitetutorial.net/download-install-sqlite/)

In [1]:
import sqlite3

## Import of JSON file
There were some problems on that step, because JSON data from your example file is not valid JSON:
- [jsonlint](http://jsonlint.com/) shows the first, obvious syntax error: single-quoted rather than double quoted strings.
- you have *NULL* where JSON requires *null*, *False* instead of *false*, *True* instead of *true*.
- missed empty value in the line 6890: 
> 'user_id': ,

All that I found during the experiments:
### Try 1: 
```python
filename = "BI_Developer_Data_Engineer_dataset.json"
with open(filename, 'rt', encoding="utf-8", errors='ignore') as f:
    json_content = json.load(f)
```
failed because [JSON requires double quotes](https://stackoverflow.com/questions/14347047/single-versus-double-quotes-in-json-loads-in-python), but getting single in the source 


### Try 2: using ast.literal_eval
```python
json_content = ast.literal_eval(json_string)
```
Works better, but failed with error anyway:
> File "<unknown>", line 6890
> {'session_id': '9d6996f6-e9db-443d-8caf-ec88de5cd1c5', 'user_ip': '159.151.92.206', 'user_server_region': 'US', 'server_version': '0.1.220', 'player_name': 'qdpwlnnrszh', 'login_attempt_id': '5306454a-d282-486f-93c7-43ddfff84dcc', 'event_name': 'session_started', 'event_timestamp': '2018-05-29 21:13:38', 'user_device_country': 'US', **'user_id': ,** 'user_type': 'PLAYER', 'client_version': '1.3.0', 'is_premium': False, 'platform': 'ANDROID_MOBILE', 'user_is_spender': False},
    
Seems, that problem with not specified **user_id** value

### Try3: replace missed empty falue in string
```python
json_content = ast.literal_eval(json_string.replace("': ,", "': '',"))
```
Failed with:
> ValueError: malformed node or string: <_ast.Name object at 0x000001EFB42732E8>

Seems, that because of not supported literal structures. From the [documentation](https://docs.python.org/2/library/ast.html#ast.literal_eval) for *ast.literal_eval()*:
> Safely evaluate an expression node or a string containing a Python expression. **The string or node provided may only consist of the following Python literal structures: strings, numbers, tuples, lists, dicts, booleans, and None.**

So, for example, *Decimal* isn't on the list of things allowed by *ast.literal_eval()*

Also, 

Finally, I could succesfully transform JSON data from file to the list of dictionaries, using intermediate string transformations: 

In [2]:
import json
filename = "BI_Developer_Data_Engineer_dataset.json"
with open(filename, 'rt', encoding="utf-8", errors='ignore') as f:
    json_string = f.read()
    
    json_string= json_string.replace('\n', ' ')
    json_string= json_string.replace("': ,", "': null,")
    json_string= json_string.replace("'", "\"")
    json_string= json_string.replace('False', 'false')
    json_string= json_string.replace('True', 'true')
    json_string= json_string.replace('NULL', 'null')
    
    json_content = json.loads(json_string)
    

    print(type(json_content))
    print(type(json_content[0]))
    print(len(json_content))

<class 'list'>
<class 'dict'>
7089


In [3]:
json_content[0]

{'session_id': '4e67da00-9c43-494d-ab96-9b6834eb6766',
 'user_ip': '49.103.200.95',
 'user_server_region': 'US',
 'server_version': '0.1.220',
 'player_name': 'xamnmngrgap',
 'login_attempt_id': '0af75d2a-0dc4-4ca3-baa5-6b57c6a4e42c',
 'event_name': 'session_started',
 'event_timestamp': '2018-06-06 00:24:49.899022',
 'user_device_country': 'US',
 'user_id': '2d2c3fa9-3a4d-4b52-a881-5cf0fe038c9d',
 'user_type': 'PLAYER',
 'client_version': '1.3.0',
 'is_premium': False,
 'platform': 'ANDROID_MOBILE',
 'user_is_spender': False}

### Convert imported list of dictionaries to a pandas DataFrame

In [4]:
import pandas as pd

df = pd.DataFrame(json_content)

In [5]:
df.head()

,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,product_name,real_currency_amount,real_currency_type,server_version,session_id,usd_cost,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,1.3.0,session_started,2018-06-06 00:24:49.899022,False,0af75d2a-0dc4-4ca3-baa5-6b57c6a4e42c,ANDROID_MOBILE,xamnmngrgap,NaN,NaN,NaN,0.1.220,4e67da00-9c43-494d-ab96-9b6834eb6766,NaN,US,2d2c3fa9-3a4d-4b52-a881-5cf0fe038c9d,49.103.200.95,False,US,PLAYER
1,1.3.0,session_started,2018-01-18 19:03:35.146217,False,9fb6bae4-27b7-4a09-97be-e0bffa5f37ae,ANDROID_MOBILE,rtnbyrpi,NaN,NaN,NaN,0.1.220,a5e9fd91-4ee2-4293-8dcb-88e9f428c2b2,NaN,US,a20abd5a-1f86-4e9a-a8bd-a6adeae828d8,18.228.191.236,False,US,PLAYER
2,1.3.0,session_started,2018-01-21 08:46:56.496430,False,794eee54-f93a-488c-a2ae-f1ed542021a6,IOS_TABLET,rgxdzxucap,NaN,NaN,NaN,0.1.220,9caae054-1fa1-44f2-b7ae-dd92c9ea1bad,NaN,US,4a9b7f36-65b6-4a2a-8c13-9a0201c8ca82,109.242.87.2,False,US,PLAYER
3,1.3.0,session_started,2018-01-11 22:41:47.391570,False,a3b309d7-7bee-43f7-97fd-ddb8f663999d,IOS_MOBILE,yfjsbgt,NaN,NaN,NaN,0.1.220,bd4ed50a-3de8-4490-9de8-5aa0da871ea2,NaN,US,3307b0e7-8259-451a-b9c5-a33bec0d33cb,139.197.237.81,False,US,PLAYER
4,1.3.0,session_started,2018-10-22 06:54:45.551663,False,15ae2c8c-9651-4262-acaa-b927b44c2638,IOS_TABLET,svmej,NaN,NaN,NaN,0.1.220,1f3dde8f-d574-458d-9bbd-fd3dcd273b6f,NaN,US,f7b17435-9131-4e31-bdff-e1ecf1fd15f9,78.238.1.65,False,US,PLAYER


In [6]:
df.columns

Index(['client_version', 'event_name', 'event_timestamp', 'is_premium',
       'login_attempt_id', 'platform', 'player_name', 'product_name',
       'real_currency_amount', 'real_currency_type', 'server_version',
       'session_id', 'usd_cost', 'user_device_country', 'user_id', 'user_ip',
       'user_is_spender', 'user_server_region', 'user_type'],
      dtype='object')

Then we need to created Database and Table there with the same structure

## [Creating a New Database](http://www.sqlitetutorial.net/sqlite-python/)
When you connect to an SQLite database file that does not exist, SQLite creates a new database for you.
To create a database, first, you have to create a `Connection` object that represents the database using the connect() function of the sqlite3 module.
The following Python program creates a new database file `test.db`.

In [7]:
from sqlite3 import Error
 
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [8]:
db_file = "test.db" 
create_connection(db_file)

2.6.0


## Creating of the DB Table with the same structure as DataFrame

In [9]:
def execute_sql(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [10]:
#conn.commit()
#conn.close()

sql_drop_table = """drop table [json]"""

conn = sqlite3.connect(db_file)
execute_sql(conn, sql_drop_table)    
conn.commit()
conn.close()

In [11]:
#conn.commit()
#conn.close()

sql_delete_from_table = """delete from [json]"""

conn = sqlite3.connect(db_file)
execute_sql(conn, sql_delete_from_table) 
conn.commit()
conn.close()

no such table: json


In [12]:
sql_create_table = """
    CREATE TABLE [json](
        --[id] [bigint] IDENTITY(1,1) NOT NULL, 
        [client_version] [nvarchar](20) NULL
        , [event_name] [nvarchar](20) NULL
        , [event_timestamp] datetime NULL
        , [is_premium] int NULL -- SQLite does not have boolean column type. Either store as nvarchar or convert to int?
        , [login_attempt_id] [nvarchar](100) NULL
        , [platform] [nvarchar](50) NULL
        , [player_name] [nvarchar](100) NULL
        , [product_name] [nvarchar](100) NULL
        , [real_currency_amount_centum] int NULL --need to replace , to . in DataFrame and multiply to 100
        , [real_currency_type] [nvarchar](5) NULL
        , [server_version] [nvarchar](20) NULL
        , [session_id] [nvarchar](100) NULL
        , [usd_cost_centum] int NULL --need to replace , to . in DataFrame and multiply to 100
        , [user_device_country] [nvarchar](10) NULL
        , [user_id] [nvarchar](10) NULL
        , [user_ip] [nvarchar](20) NULL
        , [user_is_spender] int NULL -- SQLite does not have boolean column type. Either store as nvarchar or convert to int?
        , [user_server_region] [nvarchar](10) NULL
        , [user_type] [nvarchar](20) NULL
        
        )
    """

conn = sqlite3.connect(db_file)
execute_sql(conn, sql_create_table)    
conn.commit()
conn.close()

## Transform some columns in DataFrame and write it to DataBase

In [21]:
import numpy as np

df2 = df.copy()#.loc[df['real_currency_amount'].notnull()]
#df_transformed = df2.loc[df2.real_currency_amount.str.contains(".", na=False)].iloc[0:2,:]

df_transformed = df2#.iloc[0:5,:]

#df_transformed.head()

In [22]:
df.loc[df['real_currency_amount'].notnull()].head()
#df.loc[df['user_type'].notnull()].user_type.unique()

,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,product_name,real_currency_amount,real_currency_type,server_version,session_id,usd_cost,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
56,1.3.0,session_started,2018-06-17 16:25:04.804175,False,NaN,IOS_TABLET,ycczyvtszvg,gems_small,"4,99",EUR,0.1.220,NaN,"4,99",US,18bcf1ba-ab61-4e59-90d2-26ce8630b850,251.99.219.57,True,US,PLAYER
215,1.3.0,session_started,2018-09-22 08:17:36.399105,False,NaN,ANDROID_MOBILE,osotpnpyjuw,gems_small,4.99,USD,0.1.220,NaN,4.99,US,b9c338ea-f685-49b1-b85e-fb8f5189bc76,82.14.175.53,True,US,PLAYER
234,1.3.0,session_started,2018-12-08 01:42:35.666831,False,NaN,IOS_MOBILE,nhtrujhrc,gems_small,"4,99",EUR,0.1.220,NaN,"4,99",US,8a5b16bd-c2a4-4687-93ec-43ea93829660,234.157.9.75,True,US,PLAYER
365,1.3.0,session_started,2018-10-20 13:34:44.582172,False,NaN,ANDROID_MOBILE,daeyoew,gems_medium,9.99,USD,0.1.220,NaN,9.99,US,e6cc705e-96aa-49ed-9cc3-3c9b6ea4da03,120.13.241.204,True,US,PLAYER
372,1.3.0,session_started,2018-10-18 10:07:21.420742,False,NaN,ANDROID_MOBILE,daeyoew,gems_small,"4,99",EUR,0.1.220,NaN,"4,99",US,e6cc705e-96aa-49ed-9cc3-3c9b6ea4da03,120.13.241.204,True,US,PLAYER


Problem: *real_currency_amount* and *usd_cost* - sometimes float delimeter is *dot*, sometimes - *comma*.

We need to:
- replace ',' to '.' in DataFrame for both amount columns
- transform strings to the float values 
- multiply to 100
- transfrom floats to integer, since it is better to store money in cents in SQLite Database

In [15]:
import warnings
warnings.filterwarnings('ignore')

df_transformed["real_currency_amount"] = df_transformed["real_currency_amount"].astype(str)
df_transformed["real_currency_amount"] = df_transformed["real_currency_amount"].str.replace(',', '.').astype(float)
df_transformed["real_currency_amount"].replace(np.NaN, 0, inplace=True)
df_transformed["real_currency_amount"] = df_transformed["real_currency_amount"]*100
df_transformed["real_currency_amount"] = df_transformed["real_currency_amount"].astype(int)

df_transformed["usd_cost"] = df_transformed["usd_cost"].astype(str)
df_transformed["usd_cost"] = df_transformed["usd_cost"].str.replace(',', '.').astype(float)
df_transformed["usd_cost"].replace(np.NaN, 0, inplace=True)
df_transformed["usd_cost"] = df_transformed["usd_cost"]*100
df_transformed["usd_cost"] = df_transformed["usd_cost"].astype(int)

df_transformed.head()

,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,product_name,real_currency_amount,real_currency_type,server_version,session_id,usd_cost,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,1.3.0,session_started,2018-06-06 00:24:49.899022,False,0af75d2a-0dc4-4ca3-baa5-6b57c6a4e42c,ANDROID_MOBILE,xamnmngrgap,NaN,0,NaN,0.1.220,4e67da00-9c43-494d-ab96-9b6834eb6766,0,US,2d2c3fa9-3a4d-4b52-a881-5cf0fe038c9d,49.103.200.95,False,US,PLAYER
1,1.3.0,session_started,2018-01-18 19:03:35.146217,False,9fb6bae4-27b7-4a09-97be-e0bffa5f37ae,ANDROID_MOBILE,rtnbyrpi,NaN,0,NaN,0.1.220,a5e9fd91-4ee2-4293-8dcb-88e9f428c2b2,0,US,a20abd5a-1f86-4e9a-a8bd-a6adeae828d8,18.228.191.236,False,US,PLAYER
2,1.3.0,session_started,2018-01-21 08:46:56.496430,False,794eee54-f93a-488c-a2ae-f1ed542021a6,IOS_TABLET,rgxdzxucap,NaN,0,NaN,0.1.220,9caae054-1fa1-44f2-b7ae-dd92c9ea1bad,0,US,4a9b7f36-65b6-4a2a-8c13-9a0201c8ca82,109.242.87.2,False,US,PLAYER
3,1.3.0,session_started,2018-01-11 22:41:47.391570,False,a3b309d7-7bee-43f7-97fd-ddb8f663999d,IOS_MOBILE,yfjsbgt,NaN,0,NaN,0.1.220,bd4ed50a-3de8-4490-9de8-5aa0da871ea2,0,US,3307b0e7-8259-451a-b9c5-a33bec0d33cb,139.197.237.81,False,US,PLAYER
4,1.3.0,session_started,2018-10-22 06:54:45.551663,False,15ae2c8c-9651-4262-acaa-b927b44c2638,IOS_TABLET,svmej,NaN,0,NaN,0.1.220,1f3dde8f-d574-458d-9bbd-fd3dcd273b6f,0,US,f7b17435-9131-4e31-bdff-e1ecf1fd15f9,78.238.1.65,False,US,PLAYER


for the case if it will be necessary to replace all NaN
```python
for column in df_transformed:
    print(column)
    df_transformed[column].replace(np.NaN, '', inplace=True)
df_transformed.head()
```

Rename 2 transformed to cents columns:

In [16]:
df_transformed.rename(columns={'real_currency_amount':'real_currency_amount_centum'}, inplace=True)
df_transformed.rename(columns={'usd_cost':'usd_cost_centum'}, inplace=True)


In [17]:
print(df_transformed.dtypes)
#df_transformed.values.tolist()

client_version                 object
event_name                     object
event_timestamp                object
is_premium                       bool
login_attempt_id               object
platform                       object
player_name                    object
product_name                   object
real_currency_amount_centum     int32
real_currency_type             object
server_version                 object
session_id                     object
usd_cost_centum                 int32
user_device_country            object
user_id                        object
user_ip                        object
user_is_spender                  bool
user_server_region             object
user_type                      object
dtype: object


### Write to DB

In [18]:
conn = sqlite3.connect(db_file)
print(conn)

sql = ''' 
    INSERT INTO json
        ([client_version], [event_name], [event_timestamp], [is_premium], [login_attempt_id], [platform], [player_name]
        , [product_name] , [real_currency_amount_centum], [real_currency_type], [server_version], [session_id], [usd_cost_centum], [user_device_country]
        , [user_id], [user_ip], [user_is_spender], [user_server_region], [user_type]
        )
              VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    '''
cur = conn.cursor()
cur.executemany(sql, df_transformed.values.tolist())
print(cur.lastrowid)
conn.commit()
conn.close()

None


## Check the consistency of import

In [19]:
print(len(df_transformed))
print(df_transformed['real_currency_amount_centum'].sum() )
print(df_transformed['usd_cost_centum'].sum() )
df_transformed.head(3)

7089
88911
88911


,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,product_name,real_currency_amount_centum,real_currency_type,server_version,session_id,usd_cost_centum,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,1.3.0,session_started,2018-06-06 00:24:49.899022,False,0af75d2a-0dc4-4ca3-baa5-6b57c6a4e42c,ANDROID_MOBILE,xamnmngrgap,NaN,0,NaN,0.1.220,4e67da00-9c43-494d-ab96-9b6834eb6766,0,US,2d2c3fa9-3a4d-4b52-a881-5cf0fe038c9d,49.103.200.95,False,US,PLAYER
1,1.3.0,session_started,2018-01-18 19:03:35.146217,False,9fb6bae4-27b7-4a09-97be-e0bffa5f37ae,ANDROID_MOBILE,rtnbyrpi,NaN,0,NaN,0.1.220,a5e9fd91-4ee2-4293-8dcb-88e9f428c2b2,0,US,a20abd5a-1f86-4e9a-a8bd-a6adeae828d8,18.228.191.236,False,US,PLAYER
2,1.3.0,session_started,2018-01-21 08:46:56.496430,False,794eee54-f93a-488c-a2ae-f1ed542021a6,IOS_TABLET,rgxdzxucap,NaN,0,NaN,0.1.220,9caae054-1fa1-44f2-b7ae-dd92c9ea1bad,0,US,4a9b7f36-65b6-4a2a-8c13-9a0201c8ca82,109.242.87.2,False,US,PLAYER


In [20]:
sql_select = """
    select 
            count(*) as rows_count 
            , sum([real_currency_amount_centum]) as [real_currency_amount_centum_sum]
            , sum([usd_cost_centum]) as [usd_cost_centum_sum]
        from [json]
    """

conn = sqlite3.connect(db_file)
cur = conn.cursor()
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)
conn.commit()
conn.close()

(7089, 88911, 88911)


## Analysis 

### Overview of distinct values counts
```mysql

select 
        count(*) as rows_count
        , count(distinct ifnull(client_version,'NULL')) client_version_distinct_count
        , count(distinct ifnull(event_name,'NULL')) event_name_distinct_count
        , count(distinct ifnull(is_premium,'NULL')) is_premium_distinct_count
        , count(distinct ifnull(platform,'NULL')) platform_distinct_count
        , count(distinct ifnull(player_name,'NULL')) player_name_distinct_count
        , count(distinct ifnull(product_name,'NULL')) product_name_distinct_count
        , count(distinct ifnull(server_version,'NULL')) server_version_distinct_count
        , count(distinct ifnull(real_currency_type,'NULL')) real_currency_type_distinct_count
        , count(distinct ifnull(session_id,'NULL')) session_id_distinct_count
        , count(distinct ifnull(user_device_country,'NULL')) user_device_country_distinct_count
        , count(distinct ifnull(user_id,'NULL')) user_id_distinct_count
        , count(distinct ifnull(user_ip,'NULL')) user_ip_distinct_count
        , count(distinct ifnull(user_is_spender,'NULL')) user_is_spender_distinct_count
        , count(distinct ifnull(user_server_region,'NULL')) user_server_region_distinct_count
        , count(distinct ifnull(user_type,'NULL')) user_type_distinct_count        
    from json

```

rows_count  |  client_version_distinct_count  |  event_name_distinct_count  |  is_premium_distinct_count  
----  |  -  |  -  |  -   
7089  |  1  |  1  |  1   

 player_name_distinct_count  |  product_name_distinct_count  |  server_version_distinct_count  |  real_currency_type_distinct_count 
----  |  -  |  -  |  -  
  1000  |  4  |  1  |  5
  
user_device_country_distinct_count  |  user_id_distinct_count  |  user_ip_distinct_count  |  user_is_spender_distinct_count  |  
----  |  -  |  -  |  -  
  1  |  1002  |  1000  |  2 
  
  
 user_server_region_distinct_count  |  user_type_distinct_count |  platform_distinct_count |  session_id_distinct_count 
---  | --   |  -  |  - 
  1  |  1 |  3  |  7001
  
**Columns with the only one unique value**:
- client_version : 1.3.0
- event_name : session_started
- is_premium : 0 (what means False)
- server_version: 0.1.220
- user_device_country : US
- user_server_region : US
- user_type : PLAYER

**product_name** is not specified for the most of rows
```mysql
select
        product_name, count(*)
    from json
    group by product_name  
    order by 2 desc
```
 product_name |  count 
---  | --  
NULL | 7000
gems_small | 56
gems_medium | 26
gems_large | 7

**session_id** is not specified for 89 rows
```mysql
select
        ifnull(session_id,'NULL'), count(*)
    from json
    group by ifnull(session_id,'NULL')  
    order by 2 desc
```
 session_id |  count 
---  | --  
NULL | 89
... | ...


**real_currency_type** is not specified for 1 row with existining amount
```mysql
select
        ifnull(real_currency_type,'NULL'), count(*), sum(real_currency_amount_centum)
    from json
    group by ifnull(real_currency_type,'NULL')  
    order by 2 desc
```
 real_currency_type |  count | real_currency_amount_centum_sum
---  | -- | --  
NULL | 7000 | 0
EUR | 32 | 37968
USD | 30 | 24470
RUB | 26 | 25974
 | 1 | 499

and at the same time there is no rows with the different **real_currency_amount_centum** and **usd_cost_centum** amounts, what means that conversion of the currencies does not work
```mysql
SELECT     
        *
    FROM json
    where real_currency_amount_centum != usd_cost_centum
```




**platforms** - there is no users who played on the different platforms. That also can mean that even for the same users on the different paltforms different user_id's created:
```mysql
select user_id, count(distinct platform) platforms_cnt
    from json
    group by user_id
    having count(distinct platform) > 1
    order by 2 desc;
```

Each user_id played only from the one unique IP address and most of them had 7 sessions. For 2 sessions user_id missed:
```mysql
select 
        platform, user_id, user_is_spender 
        , count(distinct user_ip)
        , count(distinct session_id)
    from json
    group by platform, user_id, user_is_spender 
    order by 4 desc, 5 desc
```


3 different **platforms** have nearly the same sessions counts, but **IOS_MOBILE** has 50% better paid sessions conversion than others (1.6% vs 1%):
```mysql
select
        ifnull(platform,'NULL') platform
        , count(*) sessions_cnt 
        , sum(paid_session) paid_session
        , 1.0 * sum(paid_session) / count(*) as paid_sessions_conversion
    from 
        (select
                case when real_currency_amount_centum > 0 then 1 else 0 end as paid_session,
                *
            from json
        ) t
    group by ifnull(platform,'NULL')
    order by 4 desc;
```
platform  |  sessions_cnt  |  paid_session  |  paid_sessions_conversion
--- | --- | --- | ---
IOS_MOBILE  |  2284  |  37  |  0.0161996497373
ANDROID_MOBILE  |  2555  |  28  |  0.01095890410959
IOS_TABLET  |  2250  |  24  |  0.01066666666667


3 different **platforms** have nearly the same unique users counts, but **IOS_MOBILE** again has 50% better spenders conversion than others (6.2% vs 4.1%):
```mysql
select
        platform
        , count(distinct user_id) user_id_count
        , sum(user_is_spender) spender_users_count
        , 1.0 * sum(user_is_spender) / count(distinct user_id)  spender_users_conversion
    from
        (
        select 
                platform, user_id, user_is_spender 
                , count(distinct user_ip) as distinct_user_ip
                , count(distinct session_id) as distinct_session_id
            from json
            group by platform, user_id, user_is_spender 
        ) t
     group by platform   
     order by 4 desc
```
platform  |  user_id_count  |  spender_users_count  |  spender_users_conversion
---  |  ---  |  ---  |  ---
IOS_MOBILE  |  321  |  20  |  0.06230529595016
ANDROID_MOBILE  |  361  |  15  |  0.0415512465374
IOS_TABLET  |  319  |  13  |  0.04075235109718

But in the same time, splitting of earned amounts per currency and platform shows that, unexpectedly, **Android Mobile** platform earns more money (in average and in total, except total leading of iPhones in Rubles): 
```mysql
select
        ifnull(real_currency_type,'NULL') as real_currency_type
        , ifnull(platform,'NULL') as platform
        , count(*) as cnt
        --, sum(real_currency_amount_centum) sum_real_currency_amount_centum
        , 1.0 * sum(real_currency_amount_centum) / 100 as sum_real_currency_amount  
        , 1.0 * max(real_currency_amount_centum) / 100  as max_real_currency_amount
        , 1.0 * min(real_currency_amount_centum) / 100  as min_real_currency_amount
        , 1.0 * avg(real_currency_amount_centum) / 100  as avg_real_currency_amount
    from json
    group by ifnull(platform,'NULL'), ifnull(real_currency_type,'NULL')  
    order by 1, 4 desc
```
real_currency_type  |  platform  |  cnt  |  sum_real_currency_amount  |  max_real_currency_amount  |  min_real_currency_amount  |  avg_real_currency_amount
--  |  --  |  --  | --  |  --  |  --  |  --  
  |  IOS_MOBILE  |  1  |  4.99  |  4.99  |  4.99  |  4.99
EUR  |  ANDROID_MOBILE  |  13  |  219.87  |  49.99  |  4.99  |  16.91
EUR  |  IOS_TABLET  |  6  |  84.94  |  49.99  |  4.99  |  14.16
EUR  |  IOS_MOBILE  |  13  |  74.87  |  9.99  |  4.99  |  5.76
NULL  |  ANDROID_MOBILE  |  2527  |  0  |  0  |  0  |  0
NULL  |  IOS_MOBILE  |  2247  |  0  |  0  |  0  |  0
NULL  |  IOS_TABLET  |  2226  |  0  |  0  |  0  |  0
RUB  |  IOS_MOBILE  |  11  |  119.89  |  49.99  |  4.99  |  10.90
RUB  |  ANDROID_MOBILE  |  5  |  69.95  |  49.99  |  4.99  |  13.99
RUB  |  IOS_TABLET  |  10  |  69.9  |  9.99  |  4.99  |  6.99
USD  |  ANDROID_MOBILE  |  10  |  109.9  |  49.99  |  4.99  |  10.99
USD  |  IOS_MOBILE  |  12  |  79.88  |  9.99  |  4.99  |  6.66
USD  |  IOS_TABLET  |  8  |  54.92  |  9.99  |  4.99  |  6.865


Splitting of paid sessions conversions per months does not show any insights:
```mysql
select
        strftime('%Y', event_timestamp) as session_year,
        strftime('%m', event_timestamp) as session_month,
        ifnull(platform,'NULL') platform
        , count(*) sessions_cnt 
        , sum(paid_session) paid_session
        , 1.0 * sum(paid_session) / count(*) as paid_sessions_conversion
    from  
        (select
                case when real_currency_amount_centum > 0 then 1 else 0 end as paid_session,
                *
            from json
        ) t
    group by ifnull(platform,'NULL'), strftime('%Y', event_timestamp), strftime('%m', event_timestamp) 
    order by 3, 1, 2 
```

session_year  |  session_month  |  platform  |  sessions_cnt  |  paid_session  |  paid_sessions_conversion
----  | ------ | -------------- | ---------- | ---------- | ------
 NULL |  NULL  |  ANDROID_MOBILE  |  2  |  0  |  0
2018  |  01  |  ANDROID_MOBILE  |  237  |  1  |  0.0042194092827
2018  |  02  |  ANDROID_MOBILE  |  231  |  0  |  0
2018  |  03  |  ANDROID_MOBILE  |  181  |  3  |  0.01657458563536
2018  |  04  |  ANDROID_MOBILE  |  209  |  0  |  0
2018  |  05  |  ANDROID_MOBILE  |  213  |  3  |  0.01408450704225
2018  |  06  |  ANDROID_MOBILE  |  238  |  1  |  0.00420168067227
2018  |  07  |  ANDROID_MOBILE  |  255  |  4  |  0.0156862745098
2018  |  08  |  ANDROID_MOBILE  |  191  |  3  |  0.01570680628272
2018  |  09  |  ANDROID_MOBILE  |  280  |  5  |  0.01785714285714
2018  |  10  |  ANDROID_MOBILE  |  243  |  5  |  0.02057613168724
2018  |  11  |  ANDROID_MOBILE  |  191  |  3  |  0.01570680628272
2018  |  12  |  ANDROID_MOBILE  |  84  |  0  |  0
 NULL |  NULL  |  IOS_MOBILE  |  3  |  0  |  0
2018  |  01  |  IOS_MOBILE  |  223  |  6  |  0.02690582959641
2018  |  02  |  IOS_MOBILE  |  196  |  5  |  0.02551020408163
2018  |  03  |  IOS_MOBILE  |  249  |  0  |  0
2018  |  04  |  IOS_MOBILE  |  205  |  3  |  0.01463414634146
2018  |  05  |  IOS_MOBILE  |  155  |  0  |  0
2018  |  06  |  IOS_MOBILE  |  165  |  8  |  0.04848484848485
2018  |  07  |  IOS_MOBILE  |  226  |  3  |  0.01327433628319
2018  |  08  |  IOS_MOBILE  |  220  |  2  |  0.00909090909091
2018  |  09  |  IOS_MOBILE  |  182  |  0  |  0
2018  |  10  |  IOS_MOBILE  |  223  |  4  |  0.01793721973094
2018  |  11  |  IOS_MOBILE  |  189  |  4  |  0.02116402116402
2018  |  12  |  IOS_MOBILE  |  48  |  2  |  0.04166666666667
 NULL |  NULL  |  IOS_TABLET  |  2  |  0  |  0
2018  |  01  |  IOS_TABLET  |  182  |  1  |  0.00549450549451
2018  |  02  |  IOS_TABLET  |  190  |  3  |  0.01578947368421
2018  |  03  |  IOS_TABLET  |  174  |  0  |  0
2018  |  04  |  IOS_TABLET  |  142  |  1  |  0.00704225352113
2018  |  05  |  IOS_TABLET  |  190  |  3  |  0.01578947368421
2018  |  06  |  IOS_TABLET  |  211  |  6  |  0.02843601895735
2018  |  07  |  IOS_TABLET  |  200  |  1  |  0.005
2018  |  08  |  IOS_TABLET  |  262  |  0  |  0
2018  |  09  |  IOS_TABLET  |  217  |  2  |  0.00921658986175
2018  |  10  |  IOS_TABLET  |  209  |  3  |  0.01435406698565
2018  |  11  |  IOS_TABLET  |  182  |  3  |  0.01648351648352
2018  |  12  |  IOS_TABLET  |  89  |  1  |  0.01123595505618



But splitting of paid sessions conversions per session time shows, that the best conversions usually happened at the late evening - early night, from **23:00 to 02:00**:
```mysql
select
        strftime('%H', event_timestamp) as session_hour
        --, ifnull(platform,'NULL') platform
        , count(*) sessions_cnt 
        , sum(paid_session) paid_session
        , 1.0 * sum(paid_session) / count(*) as paid_sessions_conversion
    from  
        (select
                case when real_currency_amount_centum > 0 then 1 else 0 end as paid_session,
                *
            from json
        ) t
    group by strftime('%H', event_timestamp) 
    order by 1 
```
session_hour  |  sessions_cnt  |  paid_session  |  paid_sessions_conversion
--- | --- | --- | ---
 NULL |  7  |  0  |  0
00  |  329  |  11  |  0.03343465045593
01  |  319  |  8  |  0.02507836990596
02  |  297  |  4  |  0.01346801346801
03  |  274  |  3  |  0.01094890510949
04  |  275  |  3  |  0.01090909090909
05  |  295  |  0  |  0
06  |  324  |  1  |  0.00308641975309
07  |  301  |  1  |  0.00332225913621
08  |  290  |  4  |  0.01379310344828
09  |  273  |  2  |  0.00732600732601
10  |  289  |  4  |  0.01384083044983
11  |  305  |  4  |  0.01311475409836
12  |  308  |  4  |  0.01298701298701
13  |  292  |  5  |  0.01712328767123
14  |  278  |  2  |  0.00719424460432
15  |  299  |  2  |  0.0066889632107
16  |  306  |  4  |  0.01307189542484
17  |  314  |  4  |  0.01273885350318
18  |  269  |  4  |  0.01486988847584
19  |  364  |  3  |  0.00824175824176
20  |  249  |  4  |  0.01606425702811
21  |  260  |  3  |  0.01153846153846
22  |  263  |  3  |  0.01140684410646
23  |  309  |  6  |  0.01941747572816

# Summary
### Problems with source file
- single quotes in the raw data instead of double
- missed empty value in the line 6890: `'user_id': ,`
- *NULL* where JSON requires *null*, *False* instead of *false*, *True* instead of *true*.
- *real_currency_amount* and *usd_cost* - sometimes float delimeter is *dot*, sometimes - *comma*
- conversion of the currencies does not work (there is no rows with the different **real_currency_amount_centum** and **usd_cost_centum** amounts)
- not all the values exist for all the rows (see examples in the "Analysis" part)

### Analysis insights (for more detailed explanations and codes see "Analysis" part above)
- there is no users who played on the different *platforms*. That also can mean that even for the same users on the different paltforms different user_id's created
- each user_id played only from the one unique IP address and most of them had 7 sessions. For 2 sessions user_id missed.
- 3 different **platforms** have nearly the same sessions counts, but **IOS_MOBILE** has 50% better paid sessions conversion than others (1.6% vs 1%)
- 3 different **platforms** have nearly the same unique users counts, but **IOS_MOBILE** again has 50% better spenders conversion than others (6.2% vs 4.1%)
- but in the same time, splitting of earned amounts per currency and platform shows that, unexpectedly, **Android Mobile** platform *earns more money* (in average and in total, except total amount leading of iPhones in Rubles)
- splitting of paid sessions conversions per months does not show any insights. But splitting of paid sessions conversions per session time shows, that the best conversions usually happened at the late evening - early night, **from 23:00 to 02:00**